In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append("/home/acho/Sync/KiddLab/MSM/src")
from utils.stim_tools import *

sys.path.append("/home/acho/Sync/Python/sigtools")
# from sigtools.representations import *
# from sigtools.sounds import *
# from sigtools.processing import *
# from sigtools.spatialization import *

# Make practice blocks
### 07/26/21

In [ ]:
stim_df = pd.read_csv(STIM_DIR/"stimulus_database.csv")
practice_task_types = ["BUG-motion_detection"]
practice_targ_amps = [30.]

n_practice_tasks = len(practice_tasks_types)
n_practice_amps = len(practice_targ_amps)

practice_threshold = 2/3
n_practice_trials_per_task_per_amp = 20
n_practice_trials_per_block_per_amp = 20
n_practice_blocks_per_task, n_practice_blocks, \
    n_practice_trials, n_practice_trials_per_block = \
    compute_n_trials(n_practice_tasks,
                     n_practice_amps,
                     n_practice_trials_per_task_per_amp,
                     n_practice_trials_per_block_per_amp)


practice_task_order = np.array([[0]])
practice_flattened_task_order = [0]

practice_corr = 1.
while practice_corr < practice_threshold:
    practice_stim_order = \
        generate_run_stim_order(stim_df,
                                practice_task_types,
                                practice_targ_amps,
                                n_practice_trials_per_task_per_amp,
                                n_practice_trials_per_block_per_amp,
                                n_practice_blocks_per_task,
                                practice_task_order)
    block_tuple = practice_stim_order[0]
    
    # Iterate through trials
    curr_block_corr = 0
    for trial_num, (stim_num, stim, target_idx, target_pattern) \
            in enumerate(block_tuple):
        # Display trial number
        trial_txt = "Practice block\nTrial {:d} of {:d}".format(
            trial_num + 1, n_practice_trials_per_block)
        helper_text.set(text=trial_txt, pos=(0, 0.75))
        helper_text.draw()
        win.flip()

        # Play stimulus
        core.wait(0.25)
        stim.play(blocking=True)

        # Wait for subject response and display feedback
        _, correct = \
            do_detection_task(win, mouse, push_button, helper_text,
                              afc_interface, target_idx)
        curr_block_corr += correct
    practice_corr = curr_block_corr/len(block_tuple)
    
    # End of block screen
    end_block_str = \
        "End of practice block\n\n\n" \
        "Score was {:.1f}\%".format(100*practice_corr)
    helper_text.set(text=end_block_str, pos=(0, 0.1))
    helper_text.draw()
    push_button.set(text="CONTINUE")
    push_button.draw()
    win.flip()
    wait_for_push_button(win, mouse, push_button)


# New stimulus ordering function
### 07/15/21

In [ ]:
def compute_n_trials(n_tasks, n_amps, n_trials_per_task_per_amp, n_trials_per_block_per_amp):
    if n_trials_per_task_per_amp%n_trials_per_block_per_amp != 0:
        raise ValueError("invalid n_trials_per_block_per_amp")
    else:
        n_blocks_per_task = int(n_trials_per_task_per_amp/n_trials_per_block_per_amp)
    n_blocks = n_blocks_per_task*n_tasks
    n_trials = n_trials_per_task_per_amp*n_amps*n_tasks
    n_trials_per_block = int(n_trials/n_blocks)
    return n_blocks_per_task, n_blocks, n_trials, n_trials_per_block


def generate_task_order(n_tasks, n_blocks_per_task, balanced=True):
    from collections import deque
    from numpy.matlib import repmat

    latin_sq = generate_latin_square(n_tasks, balanced=balanced)
    n_repetition_per_task = len(latin_sq)
    if n_blocks_per_task%n_repetition_per_task != 0:
        raise ValueError("invalid number of trials - does not fit with balanced Latin square design")
    else:
        n_repetitions = int(n_blocks_per_task/n_repetition_per_task)
    deque_latin_sq = deque(latin_sq)
    deque_latin_sq.rotate(np.random.randint(n_repetition_per_task))
    rotated_latin_sq = np.array(deque_latin_sq)
    repeated_latin_sq = repmat(rotated_latin_sq, n_repetitions, 1)
    return repeated_latin_sq


def generate_run_stim_order(stim_df,
                            task_types,
                            targ_amps,
                            n_trials_per_task_per_amp,
                            n_trials_per_block_per_amp,
                            n_blocks_per_task,
                            repeated_latin_sq):
    from random import shuffle
    def chunks(lst, n): return [lst[i:i + n] for i in range(0, len(lst), n)]

    # Draw the stim numbers and organize into task types and target amplitudes
    stim_num_dict1 = {}
    for task in task_types:
        stim_type, _ = task.split("-")
        for amp in targ_amps:
            curr_stim_nums = \
                stim_df[(stim_df["is_target"]) &
                        (stim_df["stim_type"] == stim_type) &
                        (stim_df["amplitude"] == amp)]["stim_num"].values
            chosen_stim_nums = np.random.choice(curr_stim_nums, n_trials_per_task_per_amp, replace=False)
            list_stim_nums = chosen_stim_nums.tolist()
            chunked_stim_nums = chunks(list_stim_nums, n_trials_per_block_per_amp)
            stim_num_dict1[(task, amp)] = chunked_stim_nums

    # Organize stimulus numbers into task types and blocks
    stim_num_dict2 = {}
    for task in task_types:
        accumulator2 = []
        for block_num in range(n_blocks_per_task):
            accumulator1 = []
            for amp in targ_amps:
                accumulator1 += stim_num_dict1[(task, amp)][block_num]
            shuffle(accumulator1) # randomize target amplitudes within a block
            accumulator2.append(accumulator1)
        stim_num_dict2[task] = accumulator2

    # Organize stimulus numbers into blocks
    blocked_stim_num_order = []
    for j, row in enumerate(repeated_latin_sq):
        for i in row:
            task = task_types[i]
            blocked_stim_num_order.append(stim_num_dict2[task][j])
    
    return blocked_stim_num_order


def generate_block_tuples(block_stim_nums):
    # Pack trial information into a tuples
    block_order = []
    for stim_num in block_stim_nums:
        curr_stim_fname = "stim_" + str(stim_num).zfill(5) + ".wav"
        curr_stim_path = STIM_DIR/curr_stim_fname
        curr_stim_df = stim_df[stim_df["stim_num"] == stim_num]
        target_idx = np.where(curr_stim_df["is_target"])[0][0]
        target_sentence = curr_stim_df["pattern"].values[target_idx]
        curr_stim = SoundLoader(curr_stim_path)
        block_order.append((stim_num, curr_stim, target_idx, target_sentence))
    return block_order

In [ ]:
stim_df = pd.read_csv(STIM_DIR/"stimulus_database.csv")

subject_ID = "XX"
task_types = ["SMN-motion_detection", "BUG-motion_detection", "BUG-speech_ID"]
targ_amps = [5., 10., 15., 20., 25., 30.]
n_trials_per_task_per_amp = 120
n_trials_per_block_per_amp = 10
n_tasks = len(task_types)
n_amps = len(targ_amps)

n_blocks_per_task, n_blocks, n_trials, n_trials_per_block = \
    compute_n_trials(n_tasks, n_amps, n_trials_per_task_per_amp, n_trials_per_block_per_amp)
task_order = generate_task_order(n_tasks, n_blocks_per_task)
run_stim_order = \
    generate_run_stim_order(stim_df,
                            task_types,
                            targ_amps,
                            n_trials_per_task_per_amp,
                            n_trials_per_block_per_amp,
                            n_blocks_per_task,
                            task_order)

# for block_stim_nums in run_stim_order:
#     generate_block_tuples(block_stim_nums)

In [ ]:
from datetime import datetime   
from pytz import utc, timezone

LOCAL_TZ = timezone("America/New_York")

now = datetime.now()
local_dt = LOCAL_TZ.localize(now, is_dst=None)
now_utc = local_dt.astimezone(utc)
timestr = "UTC {:s}-{:s}-{:s} {:s}:{:s}:{:s}.{:s}".format(\
              now_utc.strftime("%Y").zfill(4),
              now_utc.strftime("%m").zfill(2),
              now_utc.strftime("%d").zfill(2),
              now_utc.strftime("%H").zfill(2),
              now_utc.strftime("%M").zfill(2),
              now_utc.strftime("%S").zfill(2),
              str(now_utc.microsecond//1000).zfill(3))

print(timestr)

# Generate amplitude modulated moving noise sources
### 06/28/21

In [ ]:
fs = 44100
talkers, sentences, snds = make_sentence(3)
gns = [ALL_WORDS_SPECT.to_Noise(len(snd)/fs, fs) for snd in snds]
smn = [gns[i]*snds[i].extract_envelope() for i in range(len(gns))]
smn = normalize_rms(smn)

In [ ]:
gns[0]*snds[0].extract_envelope()

In [ ]:
snds[1].play()
plt.plot(snds[1].data)

In [ ]:
smn[1].play()
plt.plot(smn[1].data)

In [ ]:
sentences

# Generate speech ID examples and save
### 06/08/21

**MAKE SURE LEVEL IS SET EQUAL TO THE PRESENTATION STIMULI**

**DO NOT RUN BELOW UNLESS YOU WANT TO REPLACE THE EXAMPLE STIMULI**

In [ ]:
level = 65.
ex_conds = [(3, 30), (3, 30), (3, 30), (3, 15), (3, 15), (3, 15)]
ex_target_idxs, ex_stims = zip(*[make_motion_examples(*pair) for pair in ex_conds])

for i in range(len(ex_stims)):
    ex_stims[i].save(EXMP_DIR/("speech_ID_ex" + str(i + 1) + ".wav"))

with open(EXMP_DIR/"speech_ID_ex_target_idxs.txt", "w") as fin:
    for item in ex_target_idxs:
        fin.write("{:d}\n".format(item))

Read in examples:

In [ ]:
for f in sorted(os.listdir(EXMP_DIR)):
    if f.startswith("speech_ID") and f.endswith(".txt"):
        with open(EXMP_DIR/f, "r") as fin:
            lines = fin.readlines()
ex_target_idxs = [int(i) for i in lines]
ex_stims = [SoundLoader(EXMP_DIR/f) for f in sorted(os.listdir(EXMP_DIR))
            if f.startswith("speech_ID") and
               f.endswith(".wav")]

In [ ]:
speech_ID_ex_answers = ["JANE SAW NINE GREEN HATS",
                        "MIKE FOUND NINE RED CARDS",
                        "JILL LOST NINE NEW GLOVES",
                        "BOB FOUND TEN SMALL SHOES",
                        "SAM LOST NINE RED PENS",
                        "SUE GAVE FIVE GREEN GLOVES"]

# Generate detection examples and save
### 06/08/21

**DO NOT RUN BELOW UNLESS YOU WANT TO REPLACE THE EXAMPLE STIMULI**

In [ ]:
ex_conds = [(1, 30), (1, 30), (3, 30), (3, 30), (3, 15), (3, 0)]
ex_target_idxs, ex_stims = zip(*[make_motion_examples(*pair) for pair in ex_conds])

for i in range(len(ex_stims)):
    ex_stims[i].save(EXMP_DIR/("detection_ex" + str(i + 1) + ".wav"))

with open(EXMP_DIR/"detection_ex_target_idxs.txt", "w") as fin:
    for item in ex_target_idxs:
        fin.write("{:d}\n".format(item))

Read in examples:

In [ ]:
for f in sorted(os.listdir(EXMP_DIR)):
    if f.startswith("detection") and f.endswith(".txt"):
        with open(EXMP_DIR/f, "r") as fin:
            lines = fin.readlines()
ex_target_idxs = [int(i) for i in lines]
ex_stims = [SoundLoader(EXMP_DIR/f) for f in sorted(os.listdir(EXMP_DIR))
            if f.startswith("detection") and
               f.endswith(".wav")]

# Make experimental conditions using Latin square
### 04/07/21

In [ ]:
def generate_latin_square(n, balanced=False):
    """
    Williams, E. J. (1949): Experimental designs balanced
    for the estimation of residual effects of treatments.
    """
    l = [ [((1 + j//2 if j%2 == 1 else n - j//2) + i)%n + 1
           for j in range(n)]
           for i in range(n)]
    if balanced:
        if n%2 == 1:  # Repeat reversed for odd n
            l += [seq[::-1] for seq in l]
        return np.array(l) - 1
    else:
        l = np.array(l) - 1
        l[1:] = np.random.permutation(l[1:]) # permute all rows except first
        return l


def choose_stim_for_run(stim_df,
                        n_srcs,
                        targ_amps,
                        n_trials_per_block_per_amp,
                        balanced):
    latin_sq = generate_latin_square(len(n_srcs), balanced=balanced).flatten()
    src_order_list = n_srcs[latin_sq]
    n_repetitions = len(src_order_list)
    n_draw = n_trials_per_block_per_amp*n_repetitions

    # First, randomly draw stim numbers for each condition
    conditions = [(src, amp) for src in n_srcs for amp in targ_amps]
    grouped_by_count = stim_df.groupby("stim_num").count()
    is_target = stim_df[stim_df["is_target"]]
    stim_nums_by_condition = []
    for n_src, amp in conditions:
        curr_src = set(grouped_by_count[grouped_by_count["src"] == n_src].index)
        curr_amp = set(is_target[(is_target["amplitude"] == amp).values]["stim_num"].values)
        stim_nums_by_condition.append(list(curr_src.intersection(curr_amp)))
    cond_stim_num_dict = dict(zip(conditions, stim_nums_by_condition))
    drawn_stim_nums = {(src, amp): np.random.choice(cond_stim_num_dict[(src, amp)], n_draw)
                       for src in n_srcs for amp in targ_amps}

    # Next, order the stimuli in blocks
    all_block_list = []
    for i, src in enumerate(src_order_list):
        curr_amp_stim_nums = np.array([], dtype=int)
        for amp in targ_amps:
            curr_slice = slice( i     *n_trials_per_block_per_amp,
                               (i + 1)*n_trials_per_block_per_amp)
            curr_amp_stim_nums = \
                np.append(curr_amp_stim_nums, drawn_stim_nums[(src, amp)][curr_slice])
        curr_stims = [SoundLoader(STIM_DIR/("stim_" + str(stim_num).zfill(5) + ".wav"))
                      for stim_num in curr_amp_stim_nums]
        # Build pattern items list
        curr_sub_df = stim_df.loc[(stim_df["stim_num"].isin(curr_amp_stim_nums)) &
                                  (stim_df["is_target"])]
        indices = [curr_sub_df.index[curr_sub_df["stim_num"] == stim_num]
                   for stim_num in curr_amp_stim_nums]
        pattern_items = [curr_sub_df.loc[idx.values[0]]["pattern"].split(" ") for idx in indices]

        stim_tuple = list(zip(curr_stims, curr_amp_stim_nums, pattern_items))
        np.random.shuffle(stim_tuple)
        all_block_list.append(stim_tuple)
    return all_block_list, src_order_list

In [ ]:
stim_database = pd.read_csv(STIM_DIR/"stimulus_database.csv")
n_srcs = np.array([2, 3, 4])
targ_amps = np.array([0., 5., 10., 20.])
n_trials_per_block_per_amp = 5
run_stim_order, src_order = \
    choose_stim_for_run(stim_database,
                        n_srcs,
                        targ_amps,
                        n_trials_per_block_per_amp,
                        balanced=True)

# Vary angle amplitude
### 04/06/21
rate = 2 Hz

amplitudes = 5, 10, 20 deg

In [ ]:
spatial_resolution = 2 # average samples per degree over the whole trajectory
level = 80
r = 100
elev = 0

curr_n = 3
f = 2.
traj_amplitudes = [5, 10, 20]
src_spacing = 40

talkers, sentences, snds = make_sentence(curr_n)
snds = zeropad_sounds(snds)
snds = normalize_rms(snds)
snds = [snd.make_binaural() for snd in snds]
t_dur = len(snds[0])/snds[0].fs

# Designate target
target_idx = np.random.randint(curr_n)
is_target = np.zeros(curr_n, dtype=bool)
is_target[target_idx] = True
rates = np.zeros(curr_n)
rates[target_idx] = f

# Set trajectory parameters
A = traj_amplitudes[2]
B = f
C = np.random.choice([0, 0.5])
D = np.array([src_spacing*i for i in range(curr_n)])
D = D - D.mean()
# D = np.array([-90, 0, 90])
# D = np.zeros(curr_n)

# Select target and make the trajectories
trajs = [make_circular_sinuisoidal_trajectory(\
             spatial_resolution, t_dur, r, elev,
             A, B, C, D[i])
         if target
         else
         make_circular_sinuisoidal_trajectory(\
             spatial_resolution, t_dur, r, elev,
             0, 0, 0, D[i])
         for i, target in enumerate(is_target)]

# Move each source
moved_snds = [move_sound(trajs[i], snds[i]) for i in range(curr_n)]
# stimulus = moved_snds[target_idx] + compute_attenuation(level)
combined_moved_snds = normalize_rms([sum(moved_snds)])[0]
stimulus = combined_moved_snds + compute_attenuation(level)
# stimulus = combined_moved_snds - 36

In [ ]:
stimulus.play()

In [ ]:
(moved_snds[2] - 30).play()

In [ ]:
print(target_idx)
print(len(moved_snds[target_idx])/44100)
plt.plot(traj_to_theta(trajs[target_idx]))
plt.show()

In [ ]:
(moved_snds[target_idx] - 36).play()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, ax in enumerate(axes):
    ax.plot(moved_snds[i].data, alpha=0.5)
plt.show()

In [ ]:
generate_latin_square(3)

# Standard sinusoid
### 03/29/21
* A = traj_amplitude
* B = f
* C = fraction of a cycle
* D = init_angle
* A sin(2pi B (x + C/B)) + D

In [ ]:
freq = 1
t_dur = 1
t = np.linspace(0, t_dur, 200)

A = 15
B = freq
# C = np.random.uniform(0, 1)
C = 0.25
D = 45

angular_traj = A*np.sin(2*np.pi*B*(t + C/B)) + D

plt.plot(t, angular_traj)
plt.ylim((-90, 90))
plt.show()

# One static, one moving, no cue word

function of n_srcs = 2, 3, 4

function of rate = 0.1, 0.5, 2.5

compare with static cue word? or knowing where to listen

=> the degree to which motion grabs attention?

always 30 degrees apart

n_srcs, target_idx, amplitude, rate, theta_i
2, 0, [1, 0], [-15, 15]

(random direction)

how about wiggling?

In [ ]:
n_srcs = 2
src_spacing = 40
target_idx = np.random.randint(n_srcs)
traj_amplitude = 20
target_freq = 0.5

A = traj_amplitude
B = target_freq
C = np.random.uniform(size=n_srcs)
D = np.array([src_spacing*i for i in range(n_srcs)])
D = D - D.mean()

talkers, sentences, snds = make_sentence(n_srcs)
snds = zeropad_sounds(snds)
snds = [snd.make_binaural() for snd in snds]
t_dur = len(snds[0])/snds[0].fs

trajs = [make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0,
                                              0, 0, C[i], D[i])
         if i != target_idx
         else
         make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0,
                                              A, B, C[i], D[i])
         for i in range(n_srcs)]
moved_snds = [move_sound(trajs[i], snds[i]) - 30 for i in range(n_srcs)]
stim = sum(moved_snds)
# stim.play()

# Speech-shaped noise masker/multiple maskers

In [ ]:
talkers, sentences, snds = make_sentence(5, syntax_condition="random")

target_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, target_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))

In [ ]:
talkers, sentences, snds = make_sentence(5, syntax_condition="random")
answer = sentences[0]
snds = zeropad_sounds(snds)

target_snd = snds[0]
masker1_snd = snds[1]
masker2_snd = snds[2]
masker3_snd = snds[3]
masker4_snd = snds[4]

t_dur = len(target_snd)/target_snd.fs
fs = target_snd.fs

ssn_snd = ALL_WORDS_SPECT.to_Noise(t_dur, fs)
ssn_env = ssn_snd.extract_envelope()
ssn_tfs = ssn_snd/ssn_env

masker1_env = masker1_snd.extract_envelope()
masker2_env = masker2_snd.extract_envelope()
masker3_env = masker3_snd.extract_envelope()
masker4_env = masker4_snd.extract_envelope()

# smssn1_snd = normalize_rms([masker1_env*ssn_tfs])[0]
# smssn2_snd = normalize_rms([masker2_env*ssn_tfs])[0]

smssn1_snd = normalize_rms([masker1_snd])[0]
smssn2_snd = normalize_rms([masker2_snd])[0]
smssn3_snd = normalize_rms([masker3_snd])[0]
smssn4_snd = normalize_rms([masker4_snd])[0]

target_snd = target_snd.make_binaural()
smssn1_snd = smssn1_snd.make_binaural()
smssn2_snd = smssn2_snd.make_binaural()
smssn3_snd = smssn3_snd.make_binaural()
smssn4_snd = smssn4_snd.make_binaural()

target_f = 0.
smssn1_f = 0.5
smssn2_f = 0.5
smssn3_f = 0.5
smssn4_f = 0.5
target_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, target_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))
smssn1_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, smssn1_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))
smssn2_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, smssn2_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))
smssn3_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, smssn3_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))
smssn4_trj = make_circular_sinuisoidal_trajectory(2, t_dur, 100, 0, smssn4_f,
                                                  180*(np.random.random() - 0.5),
                                                  np.random.choice([True, False]))

moved_target = move_sound(target_trj, target_snd) - 30
moved_smssn1 = move_sound(smssn1_trj, smssn1_snd) - 30
moved_smssn2 = move_sound(smssn2_trj, smssn2_snd) - 30
moved_smssn3 = move_sound(smssn3_trj, smssn3_snd) - 30
moved_smssn4 = move_sound(smssn4_trj, smssn4_snd) - 30

# summed = moved_target + moved_smssn1 + moved_smssn2 + moved_smssn3 + moved_smssn4
summed = moved_target + moved_smssn1 + moved_smssn2

# fig, axes = plt.subplots(1, 3, figsize=(15, 3))
# axes[0].plot(moved_target.data, alpha=0.5)
# axes[1].plot(moved_smssn1.data, alpha=0.5)
# axes[2].plot(moved_smssn2.data, alpha=0.5)
# plt.show()

In [ ]:
summed.play()
print(answer)

# Difference between RMS and mean of abs

In [ ]:
x1 = np.linspace(0, 100*2*pi, 10000) + 2*np.pi*np.random.rand()
# x2 = x1 - np.pi
x2 = 100*x1 + 2*np.pi*np.random.rand()

y1 = 90*np.sin(x1)
y2 = 90*np.sin(x2)
y3 = y1 - y2

print(RMS(y3) - np.mean(np.abs(y3)))

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].plot(np.abs(y3))
_ = axes[1].hist(np.abs(y3), bins=100)
plt.show()

# Verify RMS of sounds after moving (should be 1)

In [ ]:
T = 1
fs = 44100
gn1 = GaussianNoise(T, 44100).make_binaural()
gn2 = GaussianNoise(T, 44100).make_binaural()

r = 100
rate1 = 1
rate2 = 10
init_angle1 = 0
init_angle2 = 0
init_R = True
traj1 = make_circular_sinuisoidal_trajectory(2, T, r, 0, rate1, init_angle1, init_R)
traj2 = make_circular_sinuisoidal_trajectory(2, T, r, 0, rate2, init_angle2, init_R)
thetas1 = traj_to_theta(traj1)
thetas2 = traj_to_theta(traj2)

moved_gn1 = move_sound(traj1, gn1)
moved_gn2 = move_sound(traj2, gn2)

In [ ]:
print(RMS(moved_gn1.data))
print(RMS(moved_gn1.data[:, 0]))
print(RMS(moved_gn1.data[:, 1]))
plt.plot(moved_gn1.data)
plt.show()

In [ ]:
print(RMS(moved_gn2.data))
print(RMS(moved_gn2.data[:, 0]))
print(RMS(moved_gn2.data[:, 1]))
plt.plot(moved_gn2.data)
plt.show()

In [ ]:
print(RMS( (moved_gn1 + moved_gn2).data ))
print(RMS( (moved_gn1.data + moved_gn2.data)[:, 0] ))
print(RMS( (moved_gn1.data + moved_gn2.data)[:, 1] ))
plt.plot(moved_gn1.data + moved_gn2.data)
plt.show()

# Generate figures to validate trajectory
Plot ITD and ILD cues for -90 to +90 trajectory to show nonlinearity:

In [ ]:
iac_dur = 5e-3 # Analysis window = 5 ms
iac = InterauralCues(moved_gn2, iac_dur)
ILDs = iac.ILDs
ITDs = iac.ITDs
ILDs[-1] = 0
ITDs[np.where(np.abs(ITDs) > 1e-3)[0]] = 0

# ILDs = ILDs[:-1]
# ITDs = 1e6*ITDs[:-1]
ITDs = 1e6*ITDs

a = 87.5e-3
c = 343
rads2 = 2*np.pi*thetas2/360
woodworth2 = 1e6*a/c*(rads2 + np.sin(rads2))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(6, 4), constrained_layout=True)
axes[0].plot(thetas2, "k-", alpha=0.8)
axes[1].plot(ILDs, "k-", alpha=0.8)
axes[2].plot(ITDs, "k-", alpha=0.8)
# axes[2].plot(np.linspace(0, len(ITDs), len(woodworth2)), woodworth2, "k--", alpha=0.5)

axes[0].set_xlim((0, len(thetas2)))
axes[1].set_xlim((0, len(ILDs)))
axes[2].set_xlim((0, len(ITDs)))

axes[0].set_xticks([round(i*len(thetas2)/5) for i in range(6)])
axes[1].set_xticks([round(i*len(ITDs)/5) for i in range(6)])
axes[2].set_xticks([round(i*len(ILDs)/5) for i in range(6)])

axes[0].set_xticklabels([])
axes[1].set_xticklabels([])
axes[2].set_xticklabels(["{:.1f}".format(i) for i in np.arange(0, T + 0.01, 0.2)], fontsize=10)

axes[2].set_xlabel("Time [s]", fontsize=14)

axes[0].set_yticks(np.arange(-90, 91, 45))
axes[1].set_yticks(np.arange(-20, 21, 10))
axes[2].set_yticks(np.arange(-800, 801, 400))

axes[0].set_yticklabels(np.arange(-90, 91, 45), fontsize=10)
axes[1].set_yticklabels(np.arange(-20, 21, 10), fontsize=10)
axes[2].set_yticklabels(np.arange(-800, 801, 400), fontsize=10)

axes[0].set_ylim((-90, 90))
axes[1].set_ylim((-22, 22))
axes[2].set_ylim((-800, 800))

axes[0].set_ylabel(r"$\theta$", fontsize=14)
axes[1].set_ylabel("ILD [dB]", fontsize=14)
axes[2].set_ylabel("ITD [$\mu$s]", fontsize=14)

axes[0].grid(linestyle=":")
axes[1].grid(linestyle=":")
axes[2].grid(linestyle=":")

# axes[0].set_title(r"\textbf{Trajectory validation}", fontsize=20)

plt.show()
# fig.savefig("validate_synthesis.pdf")

# Level is _not normalized_ after converting from spectrum to noise for SSN
### 02/08/21

In [ ]:
test_noise = ALL_WORDS_SPECT.to_Noise(2, 44100)
print(RMS(test_noise.data))

plt.plot(test_noise.data)
plt.show()